In [ ]:
import pandas as pd

### 데이터 로드

In [ ]:
review = pd.read_csv('reviews_df.csv')
review.drop('Unnamed: 0', axis=1, inplace=True)
review.head()

,유저 ID,브랜드,상품명,피부타입,피부고민,자극도,별점,한달이상사용,재구매여부,피부
0,다산정약용,에스네이처,[8월 올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획...,건성,보습,없음,5,0,0.0,건성 여름쿨톤 모공 블랙헤드
1,말티츄,에스네이처,[8월 올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획...,건성,보습,없음,3,0,0.0,지성 여름쿨톤 모공 잡티
2,하쿠나마타타고래꼬리,에스네이처,[8월 올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획...,건성,보습,없음,5,0,0.0,트러블성 봄웜톤 모공 민감성
3,유진입니다,에스네이처,[8월 올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획...,건성,진정,없음,5,0,0.0,약건성 가을웜톤 탄력
4,m튤립,에스네이처,[8월 올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획...,건성,보습,없음,5,0,0.0,건성 쿨톤 모공 블랙헤드


### 파이프라인을 이용한 데이터 전처리 및 가공

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors

# CSV 파일 로드
review = pd.read_csv('reviews_df.csv')
top100 = pd.read_csv('top100상품정보_df.csv')

# 필요없는 칼럼 제거
review.drop('Unnamed: 0', axis=1, inplace=True)

# 가격 정보를 review 데이터프레임에 병합
merged_df = pd.merge(review, top100[['상품명', 'price']], on='상품명', how='left')

# 수치형 변수와 범주형 변수 정의
num_features = ['별점', 'price']
cat_features = ['피부타입', '피부고민', '자극도', '피부', '한달이상사용', '재구매여부']

# 수치형 변환기: 결측치 평균값으로 대체 후 스케일링
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# 범주형 변환기: One-Hot Encoding
cat_transformer = OneHotEncoder(handle_unknown='ignore')

# ColumnTransformer 생성
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)])

# 전처리 수행
df_preprocessed = preprocessor.fit_transform(merged_df)

df_preprocessed.shape


(4414, 1143)

### kNN 모델을 이용한 데이터 훈련 및 유저 쿼리 기반 추천 시스템 구현

In [24]:
import numpy as np

# 사용자 입력 기반 추천 함수
def recommend_moisturizer(skin_type, skin_concern, price_limit):
    # 사용자 입력에 맞게 데이터 필터링
    filtered_df = merged_df[(merged_df['피부타입'] == skin_type) &
                            (merged_df['피부고민'] == skin_concern) &
                            (merged_df['price'] <= price_limit)]

    if filtered_df.empty:
        return "해당 조건에 맞는 수분크림이 없습니다."

    # 필터링된 데이터를 이용해 k-NN 모델 적용
    filtered_index = filtered_df.index
    filtered_preprocessed = df_preprocessed[filtered_index]

    # n_neighbors 값을 자동 조정
    n_samples = len(filtered_index)
    n_neighbors = min(230, n_samples)  # n_samples가 n_neighbors보다 작을 경우 조정

    knn_2 = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn_2.fit(filtered_preprocessed)
    recommendations = []
    for idx in filtered_index:
        # 각 샘플에 대해 n_neighbors를 데이터의 크기에 맞게 조정
        neighbors = min(n_neighbors, n_samples)
        distances, indices = knn_2.kneighbors(df_preprocessed[idx].reshape(1, -1), n_neighbors=neighbors, return_distance=True)
        recommended_indices = indices.flatten()
        distances = distances.flatten()
        recommendations.extend(zip(distances, recommended_indices))

    # 거리(distance) 기준으로 상위 15개의 추천을 랜덤으로 3개 선택 (중복 제거)
    recommendations.sort()  # 거리순으로 정렬
    top_recommendations = []
    seen_products = set()

    for _, idx in recommendations:
        product_name = filtered_df.iloc[idx]['상품명']
        base_product_name = product_name.split('[')[-1].split(']')[-1].strip()
        if base_product_name not in seen_products:
            product_price = filtered_df.iloc[idx]['price']
            top_recommendations.append((product_name, product_price))
            seen_products.add(base_product_name)
        if len(top_recommendations) >= 10:  # 상위 10개만 선택
            break

    return top_recommendations if top_recommendations else "추천할 수 있는 수분크림이 없습니다."

# 유저 입력을 받는 함수
def get_user_input():
    skin_type = input("피부타입을 입력하세요 (복합성, 건성, 지성): ")
    while skin_type not in ['복합성', '건성', '지성']:
        print("잘못된 입력입니다. 다시 입력해주세요.")
        skin_type = input("피부타입을 입력하세요 (복합성, 건성, 지성): ")

    skin_concern = input("피부고민을 입력하세요 (보습, 진정, 주름/미백): ")
    while skin_concern not in ['보습', '진정', '주름/미백']:
        print("잘못된 입력입니다. 다시 입력해주세요.")
        skin_concern = input("피부고민을 입력하세요 (보습, 진정, 주름/미백): ")

    price_limit = input("가격대를 입력하세요 (10000 이하, 20000 이하, 30000 이하, 40000 이하, 모든 가격대): ")
    price_mapping = {'10000 이하': 10000, '20000 이하': 20000, '30000 이하': 30000, '40000 이하': 40000, '모든 가격대': float('inf')}
    while price_limit not in price_mapping:
        print("잘못된 입력입니다. 다시 입력해주세요.")
        price_limit = input("가격대를 입력하세요 (10000 이하, 20000 이하, 30000 이하, 40000 이하, 모든 가격대): ")

    return skin_type, skin_concern, price_mapping[price_limit]

# 유저의 입력을 받아 추천 실행
skin_type_input, skin_concern_input, price_limit_input = get_user_input()
recommended_products = recommend_moisturizer(skin_type_input, skin_concern_input, price_limit_input)

print("추천된 상위 10개의 수분크림:")
for product, price in recommended_products:
    print(f"{product} - 가격: {price}원")

피부타입을 입력하세요 (복합성, 건성, 지성): 건정
잘못된 입력입니다. 다시 입력해주세요.
피부타입을 입력하세요 (복합성, 건성, 지성): 건성
피부고민을 입력하세요 (보습, 진정, 주름/미백): 진정
가격대를 입력하세요 (10000 이하, 20000 이하, 30000 이하, 40000 이하, 모든 가격대): 모든 가격대
추천된 상위 10개의 수분크림:
[수분크림] 에스트라 아토베리어365 하이드로 수딩크림 60ml 기획(+하이드로에센스25ml+선크림10ml) - 가격: 29700.0원
시드물 마다가스카르 리얼 수분크림 80g - 가격: 23800.0원
[8월 올영픽/대용량/수분크림]에스트라 아토베리어365 하이드로 수딩크림 100ml 기획 (+무기자차 선크림10ml) - 가격: 32900.0원
[NEW/쫀득물떡크림]스킨푸드 블루 캐모마일 히알루로닉 수분크림 기획(70ml+30ml) - 가격: 15750.0원
[김지영PICK/1+1] 아이소이 장수진 수분크림 50ml 1+1 한정기획(50ml+50ml) - 가격: 25200.0원
어바웃미 숲 진정 수분크림 80ml 기획 (+10ml) - 가격: 16500.0원
피지오겔 DMT 페이셜 수분크림 150ml - 가격: 26900.0원
[최혜선 PICK] 라운드랩 자작나무 수분 크림 80ml 기획 (+20ml 증정) - 가격: 25600.0원
[단독/대용량] 라운드랩 자작나무 수분 크림 120ml 기획(+수분패드 4매 증정) - 가격: 42000.0원
[NEW] 브링그린 대나무 히알루 수분 크림 100mL더블기획 - 가격: 19800.0원
